<h1>Extracting region of interest from each video (person)</h1>

<h2>Imports</h2>

In [0]:
!pip install cvlib
import numpy as np
import cv2
import glob
import cvlib
from cvlib.object_detection import draw_bbox
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

     |████████████████████████████████| 10.1MB 3.2MB/s 
  Created wheel for cvlib: filename=cvlib-0.2.5-cp36-none-any.whl size=10044204 sha256=9aee3a188f4bce97f8fd8f3cfe3f5ca2c7f13b19fced8d54629386d675f32ff8
  Stored in directory: /root/.cache/pip/wheels/6a/cb/43/ba188c823836640d8f22ee1f6ff792a0c83a8b66eabf52b219
  Created wheel for progressbar: filename=progressbar-2.5-cp36-none-any.whl size=12074 sha256=90011d9fd63030f7b1e9e755d334a4e43e550aaf7588dd2c780c6ed940470ee4
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built cvlib progressbar
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2f

<h2>Helper functions</h2>

This function loads the videos from a specified path and returns them in a list.

In [0]:
def get_videos_paths(path):
    videos_paths = glob.glob(path + '/*[0-9].mp4')
    videos_paths.sort()
    return videos_paths

In [0]:
def get_video_frames(video):
    frames = []
    width = video.get(3)
    height = video.get(4)
    success, frame = video.read()
    while success:
        frames.append(frame)
        success, frame = video.read()
    return frames, width, height

Next, we implement the function that applies YOLO to find the bounding boxes of the persons in one video.

This function returns a list containing the bounding boxes of each frame in one video.
If more than one bounding box are found in one frame, the bouding boxes are merged to obtain one bounding box.

In [0]:
def yolo(frames):
    # frames = get_video_frames(video)
    bounding_boxes = []
    # print('start')
    for frame in frames:
        bboxes, label, confidence = cvlib.detect_common_objects(frame)
        bboxes_person = []
        for bbox, label in zip(bboxes, label):
            if label == 'person':
                bboxes_person.append(bbox)
        if len(bboxes_person) == 0:
            continue
        x_top_left = bboxes_person[0][0]
        y_top_left = bboxes_person[0][1]
        x_bottom_right = bboxes_person[0][2]
        y_bottom_right = bboxes_person[0][3]
        for i in range(1, len(bboxes_person)): # if more than one bbox, merge them
            x_top_left = min(x_top_left, bboxes_person[i][0])
            y_top_left = min(y_top_left, bboxes_person[i][1])
            x_bottom_right = max(x_bottom_right, bboxes_person[i][2])
            y_bottom_right = max(y_bottom_right, bboxes_person[i][3])
        bbox = [x_top_left, y_top_left, x_bottom_right, y_bottom_right]
        bounding_boxes.append(bbox)
    return bounding_boxes

This function finds the bounding box with the largest area.

In [0]:
def get_max_bounding_box(bounding_boxes):
    max_bbox = []
    max_bbox_area = -1
    '''min_bbox = []
    min_bbox_area = 1000000000000000'''
    for bbox in bounding_boxes:
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        bbox_area = width * height
        if bbox_area > max_bbox_area:
            max_bbox_area = bbox_area
            max_bbox = bbox
        '''if bbox_area < min_bbox_area:
            min_bbox_area = bbox_area
            min_bbox = bbox'''
    return max_bbox #, max_bbox_area, min_bbox, min_bbox_area

This function crops one video according to the input bounding_box. It also resizes the video to the required size.

In [0]:
def crop_video(frames, width, height, bounding_box):
    x_top_left = int(max(0, bounding_box[0] - 5))
    y_top_left = int(max(0, bounding_box[1] - 5))
    x_bottom_right = int(min(width - 1, bounding_box[2] + 5)) # video.get(3) = width
    y_bottom_right = int(min(height - 1, bounding_box[3] + 5)) # video.get(4) = height
    # frames = get_video_frames(video)
    cropped_frames = []
    for frame in frames:
        cropped_frame = frame[y_top_left:y_bottom_right, x_top_left:x_bottom_right]
        cropped_frames.append(cropped_frame)
        # cv2.imshow('cropped frame', cropped_frame)
        # cv2.waitKey()
    return cropped_frames

In [0]:
def yolo_main(videos_paths):
    min_frames = 100000000
    cropped_frames_per_video = []
    for video_path in videos_paths:
        print(video_path)
        video = cv2.VideoCapture(video_path)
        frames, width, height = get_video_frames(video)
        # frames = frames[:3] # COMMENT THIS LINE AFTER TESTING
        if len(frames) < min_frames:
            min_frames = len(frames)
        bboxes = yolo(frames)
        print(len(bboxes))
        max_bbox = get_max_bounding_box(bboxes)
        print(max_bbox)
        cropped_frames = crop_video(frames, width, height, max_bbox)
        cropped_frames_per_video.append(cropped_frames)
    return cropped_frames_per_video, min_frames

<h2>Reading videos</h2>

In [0]:
def get_paths():
  path = '/content/gdrive/My Drive/Team\'s Drive/Graduation Project/Project/Dataset/Clips'
  truthful_videos_paths = get_videos_paths(path + '/Truthful')
  deceptive_videos_paths = get_videos_paths(path + '/Deceptive')
  return truthful_videos_paths, deceptive_videos_paths

In [0]:
truthful_videos_paths, deceptive_videos_paths = get_paths()
cropped_frames_per_true, min_true = yolo_main(truthful_videos_paths)

/content/gdrive/My Drive/Team's Drive/Graduation Project/Project/Dataset/Clips/Truthful/trial_truth_001.mp4


In [0]:
cropped_frames_per_lie, min_lie = yolo_main(deceptive_videos_paths)
minn = min(min_true, min_lie)

In [0]:
import os
os.mkdir('Truthful')
for i in range(len(truthful_videos_paths)):
    folder_name = 'Truthful/trial_truth_' + str(i + 1).zfill(3)
    os.mkdir(folder_name)
    sub_vids = len(cropped_frames_per_true[i]) // minn
    print(sub_vids)
    for k in range(sub_vids):
        sub_video_num = str(k).zfill(3)
        cropped_frames_slice = cropped_frames_per_true[i][k * minn : k * minn + minn]
        print(cropped_frames_slice)
        cropped_frames_np = np.array([np.array(cropped_frames) for cropped_frames in cropped_frames_slice])
        np.save(folder_name + '/' + sub_video_num + '.npy', cropped_frames_np)
    k += 1
    remainder = len(cropped_frames_per_true[i]) % minn
    if remainder > 0.5 * minn:
        sub_video_num = str(k).zfill(3)
        missing = minn - remainder
        start_idx = len(cropped_frames_per_true[i]) - remainder - missing
        cropped_frames_slice = cropped_frames_per_true[i][start_idx : len(cropped_frames_per_true[i])]
        cropped_frames_np = np.array([np.array(cropped_frames) for cropped_frames in cropped_frames_slice])
        np.save(folder_name + '/' + sub_video_num + '.npy', cropped_frames_np)
        k += 1

In [0]:
import os
os.mkdir('Deceptive')
for i in range(len(deceptive_videos_paths)):
    folder_name = 'Deceptive/trial_lie_' + str(i + 1).zfill(3)
    os.mkdir(folder_name)
    sub_vids = len(cropped_frames_per_lie[i]) // minn
    print(sub_vids)
    for k in range(sub_vids):
        sub_video_num = str(k).zfill(3)
        cropped_frames_slice = cropped_frames_per_lie[i][k * minn : k * minn + minn]
        print(cropped_frames_slice)
        cropped_frames_np = np.array([np.array(cropped_frames) for cropped_frames in cropped_frames_slice])
        np.save(folder_name + '/' + sub_video_num + '.npy', cropped_frames_np)
    k += 1
    remainder = len(cropped_frames_per_lie[i]) % minn
    if remainder > 0.5 * minn:
        sub_video_num = str(k).zfill(3)
        missing = minn - remainder
        start_idx = len(cropped_frames_per_lie[i]) - remainder - missing
        cropped_frames_slice = cropped_frames_per_lie[i][start_idx : len(cropped_frames_per_lie[i])]
        cropped_frames_np = np.array([np.array(cropped_frames) for cropped_frames in cropped_frames_slice])
        np.save(folder_name + '/' + sub_video_num + '.npy', cropped_frames_np)
        k += 1

In [0]:
!cp /content/Truthful /content/gdrive/"My Drive"
!cp /content/Deceptive /content/gdrive/"My Drive"

^C
